# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [1]:
%pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [2]:
%pip install --upgrade plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


### Import Statements

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [4]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [5]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`?
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [6]:
df_data.shape

(4324, 9)

In [7]:
df_data['Date'] = pd.to_datetime(df_data['Date'], utc=True, errors='coerce').dt.tz_localize(None)

In [8]:
df_data.head()

,Unnamed: 0.1,Unnamed: 0,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00,Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00,Atlas V 541 | Perseverance,StatusActive,145.00,Success


## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data.

In [9]:
df_data.isnull().any()

,0
Unnamed: 0.1,False
Unnamed: 0,False
Organisation,False
Location,False
Date,True
Detail,False
Rocket_Status,False
Price,True
Mission_Status,False


In [10]:
df_data['Price'] = df_data['Price'].fillna(0)

In [11]:
df_data.isnull().any()

,0
Unnamed: 0.1,False
Unnamed: 0,False
Organisation,False
Location,False
Date,True
Detail,False
Rocket_Status,False
Price,False
Mission_Status,False


## Descriptive Statistics

In [12]:
df_data.describe()

,Unnamed: 0.1,Unnamed: 0,Date,Price
count,"4,324.00","4,324.00",4198,"4,324.00"
mean,"2,161.50","2,161.50",1987-12-23 12:59:23.639828416,34.29
min,0.00,0.00,1957-10-04 19:28:00,0.00
25%,"1,080.75","1,080.75",1972-05-05 17:21:00,0.00
50%,"2,161.50","2,161.50",1985-01-24 06:21:30,0.00
75%,"3,242.25","3,242.25",2002-10-05 00:25:30,0.00
max,"4,323.00","4,323.00",2020-08-07 05:12:00,"5,000.00"
std,"1,248.38","1,248.38",NaN,150.44


In [13]:
df_data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Organisation', 'Location', 'Date',
       'Detail', 'Rocket_Status', 'Price', 'Mission_Status'],
      dtype='object')

In [14]:
df_data = df_data.drop('Unnamed: 0.1', axis=1)

In [15]:
df_data = df_data.drop('Unnamed: 0', axis=1)

In [16]:
df_data.head(1)

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success


# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [17]:
df_data['Organisation'].count()

np.int64(4324)

In [18]:
launch_count = df_data['Organisation'].value_counts()

bar = px.bar(x = launch_count.index, y = launch_count.values, title='Number of Launch Per organisation')

bar.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned?

In [19]:
df_data.head(1)

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success


In [20]:
df_data['Rocket_Status'].value_counts()

,count
Rocket_Status,
StatusRetired,3534
StatusActive,790


# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [21]:
df_data['Mission_Status'].value_counts()

,count
Mission_Status,
Success,3879
Failure,339
Partial Failure,102
Prelaunch Failure,4


# How Expensive are the Launches?

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values).

In [22]:
df_data['Date'] = pd.to_datetime(df_data['Date'], format='mixed')

In [23]:
df_data['Date'].head()

,Date
0,2020-08-07 05:12:00
1,2020-08-06 04:01:00
2,2020-08-04 23:57:00
3,2020-07-30 21:25:00
4,2020-07-30 11:50:00


In [24]:
fig = px.histogram(df_data, x='Date', y='Price')

fig.show()

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map.
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [25]:
def get_country_code(location):
    # Manual mapping for non-standard country names
    country_mapping = {
        'Russia': 'RUS',
        'South Korea': 'KOR',
        'North Korea': 'PRK',
        'New Mexico': 'USA',  # If you have US states
        'Iran': 'IRN',
        'Syria': 'SYR',
        'Venezuela': 'VEN',
        'Bolivia': 'BOL',
        'Tanzania': 'TZA',
        'Vietnam': 'VNM',
        'Laos': 'LAO'
    }

    try:
        country_name = location.split(',')[-1].strip()

        # Check manual mapping first
        if country_name in country_mapping:
            return country_mapping[country_name]

        # Otherwise use iso3166
        return countries.get(country_name).alpha3
    except (KeyError, AttributeError):
        return None

df_data['Alpha3Code'] = df_data['Location'].apply(get_country_code)

In [26]:
df_data['Alpha3Code'].unique()

array(['USA', 'CHN', 'KAZ', 'JPN', 'ISR', 'NZL', 'RUS', None, 'FRA',
       'IRN', 'IND', 'PRK', 'KOR', 'BRA', 'KEN', 'AUS'], dtype=object)

In [27]:
launch_counts = df_data.groupby('Alpha3Code').size().reset_index(name='Number_of_Launches')

fig = px.choropleth(launch_counts, locations= 'Alpha3Code',
                    color='Number_of_Launches',
                    labels={'Number_of_Launches': 'Launches'},
                    title='Number of Launches Per country',
                    hover_name='Alpha3Code',
                    color_continuous_scale = 'matter')

fig.show()

In [28]:
df_data[df_data['Alpha3Code']== 'None']

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Alpha3Code


# Use a Choropleth Map to Show the Number of Failures by Country


In [29]:
failure_by_country = df_data[df_data['Mission_Status'] == 'Failure'].groupby('Alpha3Code').size().reset_index(name='Failures Per country')

In [30]:
failure_by_country.head(2)

,Alpha3Code,Failures Per country
0,AUS,3
1,BRA,2


In [31]:
fig = px.choropleth(failure_by_country,
                    locations='Alpha3Code',
                    title="Number of Failure per country",
                    hover_name='Alpha3Code',
                    color_continuous_scale='matter',
                    color="Failures Per country")


fig.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status.

In [32]:
mission_counts = df_data.groupby(['Alpha3Code', 'Organisation', 'Mission_Status']).size().reset_index(name='Count')

In [33]:
mission_counts = mission_counts.dropna(subset=['Alpha3Code', 'Organisation', 'Mission_Status'])

In [34]:
mission_counts.head(1)

,Alpha3Code,Organisation,Mission_Status,Count
0,AUS,AMBA,Success,1


In [35]:

fig = px.sunburst(
    mission_counts,
    path=['Alpha3Code','Organisation', 'Mission_Status'],
    values='Count',
    title="Mission Success/Failure per country, organisation"
)

fig.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [36]:
df_data['Price'] = df_data['Price'].str.replace(',','').astype(float)

AttributeError: Can only use .str accessor with string values!

In [ ]:
total_money_spend = df_data.groupby('Organisation', as_index=False).agg({'Price': pd.Series.sum})

In [ ]:
df_data["Price"].dtype

In [ ]:
total_money_spend.head()

In [ ]:
fig = px.bar(total_money_spend, x='Organisation', y='Price')
fig.show()

# Chart the Number of Launches per Year

In [ ]:
# Convert to datetime first
df_data['Year'] = df_data['Date'].dt.year
launches_per_year = df_data.groupby('Year').size().reset_index(name='Launches')
launches_per_year['Cumulative_Launches'] = launches_per_year['Launches'].cumsum()


In [ ]:
launches_per_year.head()

In [ ]:
fig = px.line(launches_per_year, x='Year', y='Launches')

In [ ]:
fig.show()

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart.

In [ ]:
df_data['YearMonth' ] = df_data['Date'].dt.to_period('M').dt.to_timestamp()

In [ ]:
launches_monthly = df_data.groupby('YearMonth').size().reset_index(name='Launches')

In [ ]:
launches_monthly['Rolling Avg'] = launches_monthly['Launches'].rolling(window=12).mean()

In [ ]:
launches_monthly.head()

In [ ]:
fig = px.line(launches_monthly, x='YearMonth', y='Launches', title='Monthly Launches with rolling average')

In [ ]:
fig.add_scatter(x=launches_monthly['YearMonth'], y=launches_monthly['Rolling Avg'],mode='lines', name='Rolling avg for 12 month', line=dict(color='red', width=2))

In [ ]:

fig.update_layout(xaxis_title='Date', yaxis_title='Number of Launches')
fig.show()

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

In [ ]:
df_data.head()

In [ ]:
df_data['Month'].value_counts()

# How has the Launch Price varied Over Time?

Create a line chart that shows the average price of rocket launches over time.

In [ ]:
avg_price = df_data.groupby(['Year', 'Price'],as_index=False).agg({'Price': pd.Series.mean})

avg_price.head()

In [ ]:
fig = px.line(avg_price, x='Year', y='Price')

fig.show()

# Chart the Number of Launches over Time by the Top 10 Organisations.

How has the dominance of launches changed over time between the different players?

In [ ]:
top_10_org = df_data['Organisation'].value_counts().head(10).index

In [ ]:
df_top10 = df_data[df_data['Organisation'].isin(top_10_org)]

In [ ]:
df_top10.head()

In [ ]:
launches_by_org = df_top10.groupby(['Year', 'Organisation']).size().reset_index(name='Launches')

In [ ]:
launches_by_org.head()

In [ ]:
fig = px.line(launches_by_org, x='Year', y='Launches', color='Organisation')

In [ ]:
fig.update_layout(xaxis_title='Date', yaxis_title='Number of Launches')

fig.show()

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches.

In [37]:
df_data['Alpha3Code'].unique()

array(['USA', 'CHN', 'KAZ', 'JPN', 'ISR', 'NZL', 'RUS', None, 'FRA',
       'IRN', 'IND', 'PRK', 'KOR', 'BRA', 'KEN', 'AUS'], dtype=object)

In [38]:
former_ussr = ['RUS', 'KAZ', 'UKR', 'BLR', 'UZB', 'AZE', 'GEO',
               'TJK', 'KGZ', 'TKM', 'ARM', 'LTU', 'LVA', 'EST', 'MDA']

df_data['is_former_USSR'] = df_data['Alpha3Code'].isin(former_ussr)

In [39]:
df_data.head()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Alpha3Code,is_former_USSR
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,False
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,CHN,False
2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00,Starship Prototype | 150 Meter Hop,StatusActive,0.00,Success,USA,False
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,KAZ,True
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00,Atlas V 541 | Perseverance,StatusActive,145.00,Success,USA,False


In [40]:
df_USA_USSR = df_data[(df_data['Alpha3Code'] == "USA") | df_data['is_former_USSR']]

In [41]:
df_USA_USSR.head(1)

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Alpha3Code,is_former_USSR
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,False


In [42]:
pie_data = df_USA_USSR['is_former_USSR'].value_counts().reset_index(name='Count')

In [43]:
fig = px.pie(pie_data, values='Count', names='is_former_USSR', title='Former USSR vs US')

In [44]:
fig.show()

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

In [45]:
df_USA_USSR.shape

(3444, 9)

In [46]:
df_USA_USSR['Date'] = pd.to_datetime(df_USA_USSR['Date'], utc=True, errors='coerce').dt.tz_localize(None)


/tmp/ipython-input-4125528881.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
df_USA_USSR['Year'] = df_USA_USSR['Date'].dt.year

/tmp/ipython-input-647288207.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
launches_by_country = df_USA_USSR.groupby(['Year', 'is_former_USSR']).size().reset_index(name='Launches')

In [49]:
launches_by_country.head()

,Year,is_former_USSR,Launches
0,"1,957.00",False,1
1,"1,957.00",True,2
2,"1,958.00",False,17
3,"1,958.00",True,5
4,"1,959.00",False,16


In [50]:
launches_by_country['Country_Group'] = launches_by_country['is_former_USSR'].map({
    True: 'Former USSR',
    False: 'USA'
})

fig = px.line(launches_by_country, x='Year', y='Launches', color='Country_Group')


fig.show()

## Chart the Total Number of Mission Failures Year on Year.

In [51]:
df_data['Date'] = pd.to_datetime(df_data['Date'], utc=True, errors='coerce').dt.tz_localize(None)

In [52]:
df_data['Year'] = df_data['Date'].dt.year

In [53]:
failed_missions = df_data[df_data['Mission_Status'] == 'Failure']

In [54]:
failed_missions_by_year = failed_missions.groupby('Year').size().reset_index(name='Count')

In [55]:
fig = px.line(failed_missions_by_year, x='Year', y='Count')
fig.show()

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time?

In [57]:
failed_missions = df_data[df_data['Mission_Status'] == 'Failure']

In [60]:
failed_missions_by_year = failed_missions.groupby('Year').size().reset_index(name='Count')

In [56]:
failed_missions_by_year.head()

,Year,Count
0,"1,957.00",1
1,"1,958.00",14
2,"1,959.00",9
3,"1,960.00",19
4,"1,961.00",17


In [61]:
total_mission_by_year = df_data.groupby('Year').size().reset_index(name='Count')

In [62]:
total_mission_by_year.head()

,Year,Count
0,"1,957.00",3
1,"1,958.00",22
2,"1,959.00",20
3,"1,960.00",38
4,"1,961.00",52


In [71]:
df_failed_total = failed_missions_by_year.merge(
    total_mission_by_year,
    on='Year',
    suffixes=('_failed', '_total')
)

In [72]:
df_failed_total.head()

,Year,Count_failed,Count_total
0,"1,957.00",1,3
1,"1,958.00",14,22
2,"1,959.00",9,20
3,"1,960.00",19,38
4,"1,961.00",17,52


In [73]:
df_failed_total['Failed_percentage'] = df_failed_total['Count_failed'] / df_failed_total['Count_total']

In [74]:
df_failed_total.head()

,Year,Count_failed,Count_total,Failed_percentage
0,"1,957.00",1,3,0.33
1,"1,958.00",14,22,0.64
2,"1,959.00",9,20,0.45
3,"1,960.00",19,38,0.50
4,"1,961.00",17,52,0.33


In [75]:
fig = px.line(df_failed_total, x='Year', y='Failed_percentage', title='Failed Mission Pct by Year')

fig.show()

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches?

In [76]:
success_missions = df_data[df_data['Mission_Status'] == 'Success']

In [77]:
success_missions.head(1)

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Alpha3Code,is_former_USSR,Year
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,False,"2,020.00"


In [88]:
lead_countries = success_missions.groupby(['Year', 'Alpha3Code'], as_index=False).size()

In [97]:
lead_countries_by_year = lead_countries.sort_values(by='size', ascending=False).drop_duplicates('Year')

In [98]:
lead_countries_by_year.head()

,Year,Alpha3Code,size
82,"1,977.00",RUS,67
72,"1,975.00",RUS,64
60,"1,973.00",RUS,63
77,"1,976.00",RUS,62
87,"1,978.00",RUS,58


In [100]:
## method 2

lead_countries = success_missions.groupby(['Year', 'Alpha3Code'], as_index=False).size()
idx = lead_countries.groupby('Year')['size'].idxmax()
top_country_per_year = lead_countries.loc[idx]

In [101]:
top_country_per_year.head()

,Year,Alpha3Code,size
0,"1,957.00",KAZ,2
2,"1,958.00",USA,5
4,"1,959.00",USA,6
6,"1,960.00",USA,16
8,"1,961.00",USA,27


In [102]:
fig = px.bar(top_country_per_year, x='Year', y='size', color='Alpha3Code')

fig.show()

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020?

In [105]:
orgs_by_year = df_data.groupby(['Year', 'Organisation'], as_index=False).size()

In [106]:
orgs_by_year.head()

,Year,Organisation,size
0,"1,957.00",RVSN USSR,2
1,"1,957.00",US Navy,1
2,"1,958.00",AMBA,7
3,"1,958.00",NASA,2
4,"1,958.00",RVSN USSR,5


In [107]:
idx = orgs_by_year.groupby('Year')['size'].idxmax()

In [109]:
most_launch_orgs_by_year = orgs_by_year.loc[idx]

In [110]:
most_launch_orgs_by_year.head()

,Year,Organisation,size
0,"1,957.00",RVSN USSR,2
2,"1,958.00",AMBA,7
10,"1,959.00",US Air Force,10
15,"1,960.00",US Air Force,21
19,"1,961.00",US Air Force,23


In [111]:
fig = px.bar(most_launch_orgs_by_year, x='Year', y='size', color='Organisation')

fig.show()